<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/repeat_XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip "/content/gdrive/My Drive/Repeat/data_format2.zip" -d "/content/gdrive/My Drive/Repeat/"

Archive:  /content/gdrive/My Drive/Repeat/data_format2.zip
   creating: /content/gdrive/My Drive/Repeat/data_format2/
  inflating: /content/gdrive/My Drive/Repeat/data_format2/test_format2.csv  
  inflating: /content/gdrive/My Drive/Repeat/data_format2/train_format2.csv  


In [3]:
import gc
import pandas as pd

**数据处理**

In [6]:
# 用户行为，使用format1进行加载
# 加载全量样本
"""
user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
"""
# 加载小样本
user_log = pd.read_csv('/content/gdrive/My Drive/Repeat/data_format1_small/sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('/content/gdrive/My Drive/Repeat/data_format1_small/sample_user_info.csv')
train_data1 = pd.read_csv('/content/gdrive/My Drive/Repeat/data_format1_small/train.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Repeat/data_format1_small/test.csv')
#train_data = pd.read_csv('/content/gdrive/My Drive/Repeat/data_format2/train_format2.csv')

train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
#print(matrix)

In [7]:
print(matrix)

       user_id  merchant_id  label origin  prob
0       365952         1203    0.0  train   NaN
1        42624          946    0.0  train   NaN
2       240000         2278    0.0  train   NaN
3       177792          951    0.0  train   NaN
4       322944         1892    0.0  train   NaN
...        ...          ...    ...    ...   ...
23888    47231         1748    NaN   test   0.0
23889    59519          798    NaN   test   0.0
23890   263039          639    NaN   test   0.0
23891   263039         3954    NaN   test   0.0
23892   423551         2954    NaN   test   0.0

[23893 rows x 5 columns]


In [8]:
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
del user_info, train_data1
gc.collect()
print(matrix)

       user_id  merchant_id label origin  age_range  gender
0       365952         1203   0.0  train          0       1
1        42624          946   0.0  train          2       0
2       240000         2278   0.0  train          3       0
3       177792          951   0.0  train          0       1
4       322944         1892   0.0  train          7       0
...        ...          ...   ...    ...        ...     ...
23888    47231         1748   nan   test          0       0
23889    59519          798   nan   test          3       0
23890   263039          639   nan   test          2       1
23891   263039         3954   nan   test          2       1
23892   423551         2954   nan   test          4       0

[23893 rows x 6 columns]


In [9]:
# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
print(matrix)

       user_id  merchant_id label origin  ...      u7  u8    u9   u10
0       365952         1203   0.0  train  ...    45.0 NaN   1.0   NaN
1        42624          946   0.0  train  ...   313.0 NaN  21.0  31.0
2       240000         2278   0.0  train  ...    42.0 NaN   5.0   NaN
3       177792          951   0.0  train  ...   177.0 NaN  11.0  46.0
4       322944         1892   0.0  train  ...   147.0 NaN  25.0  14.0
...        ...          ...   ...    ...  ...     ...  ..   ...   ...
23888    47231         1748   nan   test  ...   122.0 NaN   6.0   NaN
23889    59519          798   nan   test  ...  1182.0 NaN  16.0  88.0
23890   263039          639   nan   test  ...     7.0 NaN   2.0   NaN
23891   263039         3954   nan   test  ...     7.0 NaN   2.0   NaN
23892   423551         2954   nan   test  ...   159.0 NaN  31.0   7.0

[23893 rows x 16 columns]


In [10]:
# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
print(matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


       user_id  merchant_id label origin  ...    m7     m8     m9   m10
0       365952         1203   0.0  train  ...   4.0  201.0   71.0  3518
1        42624          946   0.0  train  ...   6.0  104.0   46.0  2816
2       240000         2278   0.0  train  ...   5.0  138.0   54.0  2604
3       177792          951   0.0  train  ...   1.0   63.0   27.0  1932
4       322944         1892   0.0  train  ...  27.0  473.0  324.0  5471
...        ...          ...   ...    ...  ...   ...    ...    ...   ...
23888    47231         1748   nan   test  ...   NaN    2.0    4.0   356
23889    59519          798   nan   test  ...  12.0  304.0  253.0  8650
23890   263039          639   nan   test  ...   1.0   49.0   12.0  1943
23891   263039         3954   nan   test  ...   3.0   68.0   10.0  1592
23892   423551         2954   nan   test  ...   6.0  300.0  148.0  4497

[23893 rows x 26 columns]


In [11]:
# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
print(matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


       user_id  merchant_id label origin  ...  um6  um7  um8       um9
0       365952         1203   0.0  train  ...  NaN  1.0  NaN  0.016667
1        42624          946   0.0  train  ...  NaN  7.0  1.0  0.016667
2       240000         2278   0.0  train  ...  NaN  1.0  NaN  0.000000
3       177792          951   0.0  train  ...  NaN  1.0  NaN  0.016667
4       322944         1892   0.0  train  ...  NaN  9.0  NaN  0.016667
...        ...          ...   ...    ...  ...  ...  ...  ...       ...
23888    47231         1748   nan   test  ...  NaN  1.0  NaN  0.066667
23889    59519          798   nan   test  ...  NaN  4.0  3.0  2.783333
23890   263039          639   nan   test  ...  NaN  1.0  NaN  0.000000
23891   263039         3954   nan   test  ...  NaN  1.0  NaN  0.000000
23892   423551         2954   nan   test  ...  NaN  2.0  1.0  0.050000

[23893 rows x 35 columns]


In [12]:
#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
print(matrix)

       user_id  merchant_id label origin    u1  ...  age_7  age_8  g_0  g_1  g_2
0       365952         1203   0.0  train    46  ...      0      0    0    1    0
1        42624          946   0.0  train   365  ...      0      0    1    0    0
2       240000         2278   0.0  train    47  ...      0      0    1    0    0
3       177792          951   0.0  train   234  ...      0      0    0    1    0
4       322944         1892   0.0  train   186  ...      1      0    1    0    0
...        ...          ...   ...    ...   ...  ...    ...    ...  ...  ...  ...
23888    47231         1748   nan   test   128  ...      0      0    1    0    0
23889    59519          798   nan   test  1286  ...      0      0    1    0    0
23890   263039          639   nan   test     9  ...      0      0    0    1    0
23891   263039         3954   nan   test     9  ...      0      0    0    1    0
23892   423551         2954   nan   test   197  ...      0      0    1    0    0

[23893 rows x 47 columns]


In [13]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

213

In [22]:
train_data

,user_id,merchant_id,label,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9,r1,r2,r3,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,365952,1203,0.0,46,29,12,16,16,4.933333,45.0,0.0,1.0,0.0,1915,408,175,19,2,1639.0,4.0,201.0,71.0,3518,8,4,2,1,7.0,0.0,1.0,0.0,0.016667,0.022222,0.122636,0.142857,1,0,0,0,0,0,0,0,0,1,0
1,42624,946,0.0,365,198,46,46,45,5.866667,313.0,0.0,21.0,31.0,1965,292,320,6,3,1809.0,6.0,104.0,46.0,2816,42,14,2,1,34.0,0.0,7.0,1.0,0.016667,0.067093,0.057490,0.205882,0,1,0,0,0,0,0,0,1,0,0
2,240000,2278,0.0,47,31,14,15,17,5.833333,42.0,0.0,5.0,0.0,1125,254,72,12,2,928.0,5.0,138.0,54.0,2604,2,1,1,1,1.0,0.0,1.0,0.0,0.000000,0.119048,0.148707,1.000000,0,0,1,0,0,0,0,0,1,0,0
3,177792,951,0.0,234,105,23,35,36,5.833333,177.0,0.0,11.0,46.0,574,210,117,8,2,483.0,1.0,63.0,27.0,1932,11,4,1,1,10.0,0.0,1.0,0.0,0.016667,0.062147,0.130435,0.100000,1,0,0,0,0,0,0,0,0,1,0
4,322944,1892,0.0,186,106,34,40,39,5.866667,147.0,0.0,25.0,14.0,6852,664,554,69,3,6028.0,27.0,473.0,324.0,5471,19,10,6,1,10.0,0.0,9.0,0.0,0.016667,0.170068,0.078467,0.900000,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17832,411263,3826,0.0,40,24,13,14,15,4.916667,27.0,0.0,13.0,0.0,1331,302,18,4,3,1092.0,7.0,175.0,57.0,2154,2,1,1,1,1.0,0.0,1.0,0.0,0.000000,0.481481,0.160256,1.000000,0,0,0,0,0,1,0,0,0,1,0
17833,415103,3444,0.0,15,12,10,10,10,6.000000,10.0,0.0,5.0,0.0,489,129,17,8,2,439.0,1.0,20.0,29.0,1294,1,1,1,1,0.0,0.0,1.0,0.0,0.000000,0.500000,0.045558,0.000000,0,0,1,0,0,0,0,0,1,0,0
17834,90239,4173,0.0,16,14,8,11,12,5.833333,15.0,0.0,1.0,0.0,23229,2578,1885,23,2,21466.0,35.0,467.0,1261.0,26769,3,1,1,1,2.0,0.0,1.0,0.0,0.000000,0.066667,0.021755,0.500000,0,1,0,0,0,0,0,0,1,0,0
17835,355199,993,0.0,33,25,12,15,15,4.916667,31.0,0.0,2.0,0.0,110,23,30,5,5,99.0,0.0,8.0,3.0,213,7,2,2,1,5.0,0.0,2.0,0.0,0.000000,0.064516,0.080808,0.400000,0,0,0,0,0,1,0,0,1,0,0


In [23]:
test_data

,user_id,merchant_id,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9,r1,r2,r3,age_0,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
17837,40320,4173,24,15,10,10,11,5.783333,11.0,0.0,12.0,1.0,23229,2578,1885,23,2,21466.0,35.0,467.0,1261.0,26769,2,2,2,1,0.0,0.0,2.0,0.0,0.000000,1.090909,0.021755,0.000000,0,0,0,1,0,0,0,0,0,1,0
17838,309120,4775,72,46,14,26,26,5.816667,61.0,0.0,1.0,10.0,1913,344,237,5,2,1753.0,1.0,50.0,109.0,3602,8,1,1,1,7.0,0.0,1.0,0.0,0.166667,0.016393,0.028523,0.142857,0,1,0,0,0,0,0,0,1,0,0
17839,117120,3826,31,17,5,15,13,3.750000,30.0,0.0,1.0,0.0,1331,302,18,4,3,1092.0,7.0,175.0,57.0,2154,6,1,1,1,5.0,0.0,1.0,0.0,0.000000,0.033333,0.160256,0.200000,0,0,1,0,0,0,0,0,0,1,0
17840,127104,4048,60,42,21,22,22,5.683333,58.0,0.0,1.0,1.0,1603,545,96,6,2,1285.0,6.0,217.0,95.0,5026,4,2,1,1,3.0,0.0,1.0,0.0,0.016667,0.017241,0.168872,0.333333,0,0,0,0,0,1,0,0,0,1,0
17841,331392,1978,50,32,16,23,23,5.850000,44.0,0.0,4.0,2.0,704,195,81,10,2,623.0,2.0,57.0,22.0,1777,4,1,1,1,3.0,0.0,1.0,0.0,0.000000,0.090909,0.091493,0.333333,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,128,97,28,39,40,5.816667,122.0,0.0,6.0,0.0,72,33,14,2,1,66.0,0.0,2.0,4.0,356,9,1,1,1,8.0,0.0,1.0,0.0,0.066667,0.049180,0.030303,0.125000,1,0,0,0,0,0,0,0,1,0,0
23889,59519,798,1286,540,55,93,96,6.000000,1182.0,0.0,16.0,88.0,4892,907,109,20,2,4323.0,12.0,304.0,253.0,8650,42,15,11,1,35.0,0.0,4.0,3.0,2.783333,0.013536,0.070322,0.114286,0,0,1,0,0,0,0,0,1,0,0
23890,263039,639,9,8,7,7,7,5.783333,7.0,0.0,2.0,0.0,480,201,9,2,2,418.0,1.0,49.0,12.0,1943,2,1,1,1,1.0,0.0,1.0,0.0,0.000000,0.285714,0.117225,1.000000,0,1,0,0,0,0,0,0,0,1,0
23891,263039,3954,9,8,7,7,7,5.783333,7.0,0.0,2.0,0.0,515,171,31,7,3,434.0,3.0,68.0,10.0,1592,1,1,1,1,0.0,0.0,1.0,0.0,0.000000,0.285714,0.156682,0.000000,0,1,0,0,0,0,0,0,0,1,0


**train**

In [21]:
# 使用机器学习工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb

In [24]:
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

In [25]:
# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42    
)

In [26]:
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)

[0]	validation_0-auc:0.594219	validation_1-auc:0.551926
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.624703	validation_1-auc:0.557913
[2]	validation_0-auc:0.626591	validation_1-auc:0.558167
[3]	validation_0-auc:0.627272	validation_1-auc:0.564358
[4]	validation_0-auc:0.634417	validation_1-auc:0.572413
[5]	validation_0-auc:0.635482	validation_1-auc:0.569206
[6]	validation_0-auc:0.635336	validation_1-auc:0.567796
[7]	validation_0-auc:0.638362	validation_1-auc:0.5699
[8]	validation_0-auc:0.641447	validation_1-auc:0.57582
[9]	validation_0-auc:0.641701	validation_1-auc:0.574841
[10]	validation_0-auc:0.642916	validation_1-auc:0.573112
[11]	validation_0-auc:0.644804	validation_1-auc:0.574174
[12]	validation_0-auc:0.645101	validation_1-auc:0.576137
[13]	validation_0-auc:0.6455	validation_1-auc:0.574504
[14]	validation_0-auc:0.64584	validation_1-auc:0.575013
[15]

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=300, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=0.8, verbosity=1)

In [27]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=300, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=0.8, verbosity=1)

In [29]:
prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('/content/gdrive/My Drive/Repeat/prediction1.csv', index=False)

KeyError: ignored

In [30]:
submission

,user_id,merchant_id,prob
0,40320,4173,0.110009
1,309120,4775,0.102542
2,117120,3826,0.084361
3,127104,4048,0.085851
4,331392,1978,0.076084
...,...,...,...
6051,47231,1748,0.084062
6052,59519,798,0.137908
6053,263039,639,0.075817
6054,263039,3954,0.077279


In [31]:
submission.to_csv('/content/gdrive/My Drive/Repeat/prediction1.csv', index=False)